<h1>Asan Dopollos Model</h1>

In [97]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder


# Import datasets
network = pd.read_csv('network.csv')
inspections = pd.read_csv('inspections.csv')
pipes = pd.read_csv('sample_submission.csv')

## Data Visualization

In [98]:
network.head()

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,489616,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,63.0,7.792,0.4,0,0,False
1,489645,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,2.080,0.4,0,0,False
2,489646,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,111.173,0.4,3,0,False
3,519950,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,200.0,14.304,0.4,0,0,False
4,1332439,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,160.0,22.669,0.4,1,0,False


In [99]:
pipes.head()

,PipeId,Incidence
0,446859944,0
1,428124500,0
2,438428871,0
3,429034569,0
4,411184477,0


In [100]:
inspections.head()

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0
1,188889914,ZRV-00001406,2010,2010-10-01,22,4,0
2,188889907,ZRV-00001406,2010,2010-10-01,22,4,0
3,188673355,ZRV-00001406,2010,2010-10-01,22,4,0
4,188558922,ZRV-00001406,2010,2010-10-01,22,4,0


Merging the df with an inner join using PipeId

In [101]:
# Merge both dataframes
df = inspections.merge(
                        network,
                        how = 'inner',
                        on=['PipeId'] 
                        )

df

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
1,189311802,ZRV-00001972,2012,2012-10-01,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
2,189311802,ZRV-00001972,2014,2014-10-08,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
3,189311802,ZRV-00001972,2016,2016-10-14,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
4,189311802,ZRV-00001972,2018,2018-10-09,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,ZRV-00004581,2020,2020-12-31,24,4,0,Valencia,Betera,1993,PE,Gas natural,63.0,1.237,4.000,0,0,False
6345340,56907425,ZRV-00004581,2020,2020-12-31,24,4,0,Valencia,Betera,2000,PE,Gas natural,160.0,169.732,4.000,1,0,False
6345341,7843615,ZRV-00002112,2021,2020-12-31,24,4,0,Barcelona,Sabadell,1995,FD,Gas natural,150.0,0.361,0.025,0,0,True
6345342,56922456,ZRV-00004581,2020,2020-12-31,24,4,0,Valencia,Betera,1993,PE,Gas natural,63.0,23.306,4.000,0,0,False


## Data preparation

Dates will be converted to datetime.

In [102]:
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'])

Column Dop: MaintenaceId, Town, NumConnectionsUnder

In [103]:
df = df.drop(['MaintenanceId','NumConnectionsUnder','Town'],axis=1)
df

,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle
0,189311802,2010,2010-10-01,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
1,189311802,2012,2012-10-01,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
2,189311802,2014,2014-10-08,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
3,189311802,2016,2016-10-14,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
4,189311802,2018,2018-10-09,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,2020,2020-12-31,24,4,0,Valencia,1993,PE,Gas natural,63.0,1.237,4.000,0,False
6345340,56907425,2020,2020-12-31,24,4,0,Valencia,2000,PE,Gas natural,160.0,169.732,4.000,1,False
6345341,7843615,2021,2020-12-31,24,4,0,Barcelona,1995,FD,Gas natural,150.0,0.361,0.025,0,True
6345342,56922456,2020,2020-12-31,24,4,0,Valencia,1993,PE,Gas natural,63.0,23.306,4.000,0,False


Value standarization: Diameter, Length, Pressure

In [104]:
for column in ['Diameter', 'Length', 'Pressure']:
   df[column] = (df[column] - df[column].mean()) / df[column].std()

df.head()

,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle
0,189311802,2010,2010-10-01,24,4,0,Barcelona,2001,PE,Gas natural,0.707025,1.12697,0.166398,0,False
1,189311802,2012,2012-10-01,24,4,0,Barcelona,2001,PE,Gas natural,0.707025,1.12697,0.166398,0,False
2,189311802,2014,2014-10-08,24,4,0,Barcelona,2001,PE,Gas natural,0.707025,1.12697,0.166398,0,False
3,189311802,2016,2016-10-14,24,4,0,Barcelona,2001,PE,Gas natural,0.707025,1.12697,0.166398,0,False
4,189311802,2018,2018-10-09,24,4,0,Barcelona,2001,PE,Gas natural,0.707025,1.12697,0.166398,0,False


Severity values will be inverted, meaning that they will go from 0-3 going from no incidence to highest severity

In [105]:
inverse_sev = {4:0,3:1,2:2,1:3}
list = []
for sev in df["Severity"]:
    list.append(inverse_sev[sev]) 

df['Severity'] = list
df

,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle
0,189311802,2010,2010-10-01,24,0,0,Barcelona,2001,PE,Gas natural,0.707025,1.126970,0.166398,0,False
1,189311802,2012,2012-10-01,24,0,0,Barcelona,2001,PE,Gas natural,0.707025,1.126970,0.166398,0,False
2,189311802,2014,2014-10-08,24,0,0,Barcelona,2001,PE,Gas natural,0.707025,1.126970,0.166398,0,False
3,189311802,2016,2016-10-14,24,0,0,Barcelona,2001,PE,Gas natural,0.707025,1.126970,0.166398,0,False
4,189311802,2018,2018-10-09,24,0,0,Barcelona,2001,PE,Gas natural,0.707025,1.126970,0.166398,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,2020,2020-12-31,24,0,0,Valencia,1993,PE,Gas natural,-0.988359,-0.489225,0.166398,0,False
6345340,56907425,2020,2020-12-31,24,0,0,Valencia,2000,PE,Gas natural,0.707025,1.846408,0.166398,1,False
6345341,7843615,2021,2020-12-31,24,0,0,Barcelona,1995,FD,Gas natural,0.532243,-0.501367,-0.438394,0,True
6345342,56922456,2020,2020-12-31,24,0,0,Valencia,1993,PE,Gas natural,-0.988359,-0.183310,0.166398,0,False


## Feature Engineering

### Age of the pipe

In [106]:
age_list = []

for ind in df.index:
    age = int(df['InspectionDate'][ind].year - df['YearBuilt'][ind])
    age_list.append(age)

df['Age'] = age_list

### Previous revision severity

In [107]:
df = df.sort_values(['PipeId','InspectionDate'])
df = df.reset_index(drop=True)

prev_sev = 0
previous_severity = []

for ind in df.index:
    previous_severity.append(prev_sev)
    prev_sev = df['Severity'][ind]

df['PrevSeverity'] = previous_severity
df[df.groupby('PipeId').cumcount() == 0]['PrevSeverity'] = 0

df

C:\Users\rodri\AppData\Local\Temp\ipykernel_4348\221099879.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.groupby('PipeId').cumcount() == 0]['PrevSeverity'] = 0


,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle,Age,PrevSeverity
0,489616,2013,2013-05-08,16,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,1,0
1,489616,2015,2015-03-04,22,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,3,0
2,489616,2016,2017-01-12,22,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,5,0
3,489616,2018,2018-12-19,23,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,6,0
4,489616,2020,2020-12-05,24,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,451995260,2019,2019-10-23,24,0,0,Lleida,2003,PE,Gas natural,-0.516448,0.426301,-0.419375,2,False,16,0
6345340,451995309,2013,2013-10-07,24,0,0,Lleida,1998,PE,Gas natural,1.406152,0.771957,-0.419375,0,False,15,0
6345341,451995309,2015,2015-10-05,24,0,0,Lleida,1998,PE,Gas natural,1.406152,0.771957,-0.419375,0,False,17,0
6345342,451995309,2017,2017-10-02,24,0,0,Lleida,1998,PE,Gas natural,1.406152,0.771957,-0.419375,0,False,19,0


### Accumulated number of incidence per pipe and accumulated number of revisions

In [108]:
accum_incidence_list = [0]
incidence_count = df['Incidence'][0]
revision_count = 0
revision_list = [0]

for ind in range(1, len(df.index)):
    if df['PipeId'][ind] != df['PipeId'][ind-1]:
        incidence_count = 0
        revision_count = 0
    accum_incidence_list.append(incidence_count)
    revision_list.append(revision_count)
    incidence_count = incidence_count + df['Incidence'][ind]
    revision_count += 1

df['AcumIncidence'] = accum_incidence_list
df['AcumRevisions'] = revision_list

One hot encoding the categorical values with pandas get dummies

In [109]:
df = pd.get_dummies(df)
df.head(4)

,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,YearBuilt,Diameter,Length,Pressure,...,Material_FG,Material_FI,Material_FO,Material_PA,Material_PE,Material_PN,Material_PV,Material_ZD,GasType_Gas natural,GasType_Gas propano
0,489616,2013,2013-05-08,16,0,0,2012,-0.988359,-0.398361,-0.381338,...,0,0,0,0,1,0,0,0,1,0
1,489616,2015,2015-03-04,22,0,0,2012,-0.988359,-0.398361,-0.381338,...,0,0,0,0,1,0,0,0,1,0
2,489616,2016,2017-01-12,22,0,0,2012,-0.988359,-0.398361,-0.381338,...,0,0,0,0,1,0,0,0,1,0
3,489616,2018,2018-12-19,23,0,0,2012,-0.988359,-0.398361,-0.381338,...,0,0,0,0,1,0,0,0,1,0


The dataframe will be reindexed by means of the PipeId and Date

In [110]:
df = df.set_index(['PipeId', 'InspectionDate','Severity'])
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6345344 entries, (489616, Timestamp('2013-05-08 00:00:00'), 0) to (451995309, Timestamp('2019-10-26 00:00:00'), 0)
Data columns (total 64 columns):
 #   Column                Dtype  
---  ------                -----  
 0   InspectionYear        int64  
 1   MonthsLastRev         int64  
 2   Incidence             int64  
 3   YearBuilt             int64  
 4   Diameter              float64
 5   Length                float64
 6   Pressure              float64
 7   NumConnections        int64  
 8   BoolBridle            bool   
 9   Age                   int64  
 10  PrevSeverity          int64  
 11  AcumIncidence         int64  
 12  AcumRevisions         int64  
 13  Province_Albacete     uint8  
 14  Province_Alicante     uint8  
 15  Province_Almeria      uint8  
 16  Province_Avila        uint8  
 17  Province_Barcelona    uint8  
 18  Province_Burgos       uint8  
 19  Province_Castellón    uint8  
 20  Province_Ciudad Real  uint8

Balancing the dataframe as it quite unbalanced

In [111]:
# First we check how balanced is the dataset
Incidence = df[df.Incidence == 1]
not_Incidence = df[df.Incidence == 0]

# index property contains the indexes of the not_Incidence dataset
not_Incidence_indexes = not_Incidence.index

# get as many not_incidence indexes (randomly) as incidence occurrences
np.random.seed(42)
random_not_Incidence_indexes = np.random.choice(not_Incidence_indexes, 10*len(Incidence))

# Keep those entries in not_Incidence
not_Incidence = df.loc[random_not_Incidence_indexes]

# We will form the balanced dataset concatenating Incidence and not_Incidence
balanced_df = Incidence.append(not_Incidence)
balanced_df.sort_index()
df = balanced_df

C:\Users\rodri\AppData\Local\Temp\ipykernel_4348\2127044534.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_df = Incidence.append(not_Incidence)


,,,InspectionYear,MonthsLastRev,Incidence,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Age,...,Material_FG,Material_FI,Material_FO,Material_PA,Material_PE,Material_PN,Material_PV,Material_ZD,GasType_Gas natural,GasType_Gas propano
PipeId,InspectionDate,Severity,,,,,,,,,,,,,,,,,,,,,
489981,2016-04-28,0,2016,22,0,2012,-0.166884,0.173283,0.166398,0,False,4,...,0,0,0,0,1,0,0,0,1,0
490349,2020-05-18,0,2020,24,0,2012,1.406152,-0.351245,1.079290,0,False,8,...,0,0,0,0,1,0,0,0,1,0
490411,2018-05-15,0,2018,24,0,2012,1.406152,0.207923,1.079290,0,False,6,...,0,0,0,0,1,0,0,0,1,0
490620,2020-05-18,0,2020,24,0,2012,0.707025,-0.412112,1.079290,0,False,8,...,0,0,0,0,1,0,0,0,1,0
490635,2017-08-31,0,2017,24,0,2012,0.707025,-0.500785,-0.381338,0,False,5,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451035772,2014-10-29,0,2014,22,0,2012,1.406152,0.446609,-0.381338,1,False,2,...,0,0,0,0,1,0,0,0,1,0
451125808,2020-06-12,0,2020,25,0,2012,0.707025,-0.495268,-0.434590,0,False,8,...,0,0,0,0,1,0,0,0,1,0
451194879,2013-01-10,0,2013,12,0,2012,-0.516448,0.386241,-0.419375,3,False,1,...,0,0,0,0,1,0,0,0,1,0


Splitting in a train-test set

In [112]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Incidence'],axis=1),df['Incidence'])

## Training and testing the model

BayesianNB Classification

In [113]:
forecaster = GaussianNB()
forecaster.fit(X_train, y_train)
Incidence_pred = forecaster.predict(X_train)
print(metrics.roc_auc_score(y_train,Incidence_pred))

0.6735327849057785


## Creating prediction dataset

Merging with the PipeId from the Submission dataset

In [114]:
pipes = pipes.drop(['Incidence'],axis=1)
pred_X = network.merge(
                pipes,
                how='inner',
                on=['PipeId']
)
pred_X

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,489780,Toledo,Añover de Tajo,2009,PE,Gas natural,110.0,7.485,5.0,0,0,False
1,489790,Toledo,Añover de Tajo,2012,PE,Gas natural,110.0,4.436,4.0,0,0,False
2,489792,Toledo,Añover de Tajo,2012,PE,Gas natural,110.0,11.769,4.0,1,0,False
3,489793,Toledo,Añover de Tajo,2012,PE,Gas natural,110.0,6.172,4.0,1,0,False
4,489981,Toledo,Añover de Tajo,2012,PE,Gas natural,110.0,49.031,4.0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
909728,416223449,La Rioja,Cirueña,2011,AO,Gas natural,50.8,0.460,45.0,0,0,False
909729,416223471,La Rioja,Cirueña,2011,AO,Gas natural,50.8,0.230,45.0,0,0,False
909730,416425646,La Rioja,Nájera,2011,AO,Gas natural,254.0,1.999,45.0,0,0,False
909731,416425691,La Rioja,Nájera,2011,AO,Gas natural,254.0,3.000,45.0,0,0,False


Drop columns

In [115]:
pred_X = pred_X.drop(['Town', 'NumConnectionsUnder'],axis=1)

Standarization

In [116]:
for column in ['Diameter', 'Length', 'Pressure']:
   pred_X[column] = (pred_X[column] - pred_X[column].mean()) / pred_X[column].std()

pred_X.head()

,PipeId,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle
0,489780,Toledo,2009,PE,Gas natural,-0.111998,-0.421161,0.387956,0,False
1,489790,Toledo,2012,PE,Gas natural,-0.111998,-0.462561,0.216777,0,False
2,489792,Toledo,2012,PE,Gas natural,-0.111998,-0.362993,0.216777,1,False
3,489793,Toledo,2012,PE,Gas natural,-0.111998,-0.438989,0.216777,1,False
4,489981,Toledo,2012,PE,Gas natural,-0.111998,0.142952,0.216777,0,False


Obtaining the MonthLastRev column (we assume the next revision will be in 24 months from the last)

In [117]:
pred_X['MonthsLastRev'] = 24

With this MonthLastRev we estimate the inspection date

In [118]:
last_rev_df = df
last_rev_df.reset_index(inplace=True)
last_rev_df = last_rev_df.drop_duplicates(subset=['PipeId'], keep='last')
pred_X = pred_X.merge(last_rev_df[['PipeId', 'Severity', 'AcumRevisions', 'AcumIncidence','InspectionDate','Incidence']], on = 'PipeId', how = 'left')
pred_X.rename(columns = {'Severity':'PrevSeverity'}, inplace = True)



Inspection Date (estimated)

In [119]:
new_dates = []

for ind in pred_X.index:
    new_date = pred_X['InspectionDate'][ind]+pd.DateOffset(months=pred_X['MonthsLastRev'][ind])
    new_dates.append(new_date)

pred_X['InspectionDate'] = new_dates

Changing the NaN in Severity to 0, as some of them have never been revised. Same will happen with the accumulated values

In [122]:
pred_X['InspectionDate'] = pred_X['InspectionDate'].replace(pd.NaT, pd.to_datetime('2022-12-31'))
pred_X['PrevSeverity'] = pred_X['PrevSeverity'].replace(np.nan, 0)
pred_X['AcumRevisions'] = pred_X['AcumRevisions'].replace(np.nan, 0)
pred_X['AcumIncidence'] = pred_X['AcumIncidence'].replace(np.nan, 0)

Age of the pipe

In [126]:
age_list = []

for ind in pred_X.index:
    age = int(pred_X['InspectionDate'][ind].year - pred_X['YearBuilt'][ind])
    age_list.append(age)

pred_X['Age'] = age_list

Accumulated Incidences

In [128]:
pred_X['AcumIncidence'] = pred_X['AcumIncidence'] + pred_X['Incidence']
pred_X = pred_X.drop(['Incidence'],axis=1)

Accumulated Revisions

In [129]:
pred_X['AcumRevisions'] = pred_X['AcumRevisions'] + 1

Inspection Year

In [137]:
pred_X['InspectionYear'] = pred_X['InspectionDate'].dt.year
pred_X['PrevSeverity'] = pred_X['PrevSeverity'].astype('int')
pred_X['AcumIncidence'] = pred_X['AcumIncidence'].astype('int')
pred_X['AcumRevisions'] = pred_X['AcumRevisions'].astype('int')


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer